# Cleaning:
1. Remove admission_ids which have both icu_mortality and mortality_after_discharge as True
3. Remove row in each admission which are very close (<=5 hrs) to discharge. This is to avoid data leakage on discharge decision.
4. Add A
5. Add Y (Mortality within 7 days after discharge)
6. Add D (In-ICU mortality as a competing event)

In [1]:
import pandas as pd
import numpy as np

In [2]:
sf_mimicdata = pd.read_parquet('../Dataset/mimic-iv/from_pipeline/ltm_grided_cleaned_5000.parquet')

In [3]:
sf_mimicdata

,stay_id,grid_end,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,po2_arterial__mean__last_12h,...,unit_type,origin,los,intime,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_within_7days_after_discharge,vent_mode__hours_since_last__last_12h
0,mimic4-30002548,0 days 12:00:00,36.622223,79.230766,71.000000,225.000000,43.166668,13.384615,98.461540,306.166656,...,Cardiac,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,0
1,mimic4-30002548,1 days 00:00:00,36.944447,70.500000,79.571426,87.500000,NaN,18.250000,98.666664,NaN,...,Cardiac,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,12
2,mimic4-30002548,1 days 12:00:00,37.333332,76.500000,87.750000,38.750000,NaN,21.000000,97.500000,NaN,...,Cardiac,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,24
3,mimic4-30003202,0 days 12:00:00,37.208336,99.312500,52.571430,46.500000,NaN,18.466667,96.533333,NaN,...,Surgical,Emergency,0 days 16:56:08.999999,2188-07-03 20:38:01,2188-07-04 13:34:10,183 days 03:21:59,0,2189-01-03,0,0
4,mimic4-30003202,1 days 00:00:00,37.166668,91.400002,56.400002,95.000000,NaN,16.400000,93.599998,NaN,...,Surgical,Emergency,0 days 16:56:08.999999,2188-07-03 20:38:01,2188-07-04 13:34:10,183 days 03:21:59,0,2189-01-03,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38822,mimic4-39997955,3 days 00:00:00,36.907406,69.083336,95.916664,230.000000,NaN,21.166666,97.166664,NaN,...,Surgical,Emergency,4 days 18:20:08,2160-10-26 19:03:02,2160-10-31 13:23:10,NaT,0,NaT,0,60
38823,mimic4-39997955,3 days 12:00:00,36.777779,70.500000,89.833336,245.714279,NaN,18.166666,96.416664,NaN,...,Surgical,Emergency,4 days 18:20:08,2160-10-26 19:03:02,2160-10-31 13:23:10,NaT,0,NaT,0,72
38824,mimic4-39997955,4 days 00:00:00,36.861115,80.000000,98.416664,209.375000,NaN,19.000000,96.833336,NaN,...,Surgical,Emergency,4 days 18:20:08,2160-10-26 19:03:02,2160-10-31 13:23:10,NaT,0,NaT,0,84
38825,mimic4-39997955,4 days 12:00:00,36.629635,84.833336,94.333336,162.727280,NaN,17.333334,95.500000,NaN,...,Surgical,Emergency,4 days 18:20:08,2160-10-26 19:03:02,2160-10-31 13:23:10,NaT,0,NaT,0,96


In [4]:
sf_mimicdata["timestep"] = (
    sf_mimicdata.sort_values(["stay_id", "grid_end"])
              .groupby("stay_id")
              .cumcount()
)


In [5]:
sf_mimicdata = sf_mimicdata.rename(columns={"timestep": "t"})

In [6]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'temperature__mean__last_12h',
       'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'age', 'sex', 'height', 'weight',
       'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_within_7days_after_discharge',
       'vent_mode__hours_since_last__last_12h', 't'],
      dtype='object')

In [7]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [8]:
sf_mimicdata.stay_id.nunique()

5000

In [9]:
nan_columns = sf_mimicdata.columns[sf_mimicdata.isna().all()].tolist()
nan_columns

[]

In [ ]:
#mimicdata = mimicdata.drop(columns=nan_columns)

In [10]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'temperature__mean__last_12h',
       'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'age', 'sex', 'height', 'weight',
       'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_within_7days_after_discharge',
       'vent_mode__hours_since_last__last_12h', 't'],
      dtype='object')

In [11]:
sf_mimicdata.head(10)

,stay_id,grid_end,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,po2_arterial__mean__last_12h,...,origin,los,intime,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_within_7days_after_discharge,vent_mode__hours_since_last__last_12h,t
0,mimic4-30002548,0 days 12:00:00,36.622223,79.230766,71.000000,225.000000,43.166668,13.384615,98.461540,306.166656,...,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,0,0
1,mimic4-30002548,1 days 00:00:00,36.944447,70.500000,79.571426,87.500000,NaN,18.250000,98.666664,NaN,...,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,12,1
2,mimic4-30002548,1 days 12:00:00,37.333332,76.500000,87.750000,38.750000,NaN,21.000000,97.500000,NaN,...,Operating Room,1 days 05:36:48,2111-08-17 13:13:43,2111-08-18 18:50:31,NaT,0,NaT,0,24,2
3,mimic4-30003202,0 days 12:00:00,37.208336,99.312500,52.571430,46.500000,NaN,18.466667,96.533333,NaN,...,Emergency,0 days 16:56:08.999999,2188-07-03 20:38:01,2188-07-04 13:34:10,183 days 03:21:59,0,2189-01-03,0,0,0
4,mimic4-30003202,1 days 00:00:00,37.166668,91.400002,56.400002,95.000000,NaN,16.400000,93.599998,NaN,...,Emergency,0 days 16:56:08.999999,2188-07-03 20:38:01,2188-07-04 13:34:10,183 days 03:21:59,0,2189-01-03,0,12,1
5,mimic4-30003372,0 days 12:00:00,37.407410,109.083336,73.000000,70.699997,NaN,19.166666,98.000000,NaN,...,Other,2 days 12:39:30.999999,2170-06-10 03:15:10,2170-06-12 15:54:41,NaT,0,NaT,0,0,0
6,mimic4-30003372,1 days 00:00:00,37.597221,110.384613,69.384613,93.181816,NaN,21.153847,95.923080,NaN,...,Other,2 days 12:39:30.999999,2170-06-10 03:15:10,2170-06-12 15:54:41,NaT,0,NaT,0,12,1
7,mimic4-30003372,1 days 12:00:00,37.180557,100.083336,75.333336,131.428574,NaN,18.416666,96.000000,NaN,...,Other,2 days 12:39:30.999999,2170-06-10 03:15:10,2170-06-12 15:54:41,NaT,0,NaT,0,24,2
8,mimic4-30003372,2 days 00:00:00,36.703709,105.500000,81.272728,153.333328,NaN,19.583334,97.166664,NaN,...,Other,2 days 12:39:30.999999,2170-06-10 03:15:10,2170-06-12 15:54:41,NaT,0,NaT,0,36,3
9,mimic4-30003372,2 days 12:00:00,36.814819,95.750000,86.599998,220.000000,NaN,18.500000,97.666664,NaN,...,Other,2 days 12:39:30.999999,2170-06-10 03:15:10,2170-06-12 15:54:41,NaT,0,NaT,0,48,4


In [12]:
sf_mimicdata.iloc[0]

stay_id                                                      mimic4-30002548
grid_end                                                     0 days 12:00:00
temperature__mean__last_12h                                        36.622223
heart_rate__mean__last_12h                                         79.230766
arterial_blood_pressure_mean__mean__last_12h                            71.0
fluid_out_urine__mean__last_12h                                        225.0
pco2_arterial__mean__last_12h                                      43.166668
respiratory_rate_measured__mean__last_12h                          13.384615
o2_saturation__mean__last_12h                                       98.46154
po2_arterial__mean__last_12h                                      306.166656
bicarbonate_arterial__last__last_12h                                     NaN
activated_partial_thromboplastin_time__last__last_12h                    NaN
hemoglobin__last__last_12h                                               NaN

# Check if there are any stays with both icu_mortality=True and mortality_after_discharge=True

In [13]:
#ratio of mortality
last_rows = sf_mimicdata.groupby("stay_id").tail(1)
(last_rows["mortality_within_7days_after_discharge"] == 1).mean()

0.0356

In [14]:
# Identify admission_ids where both icu_mortality and mortality_after_discharge are True. These are admissions which have death very close to discharge (<=12 hours)
# means that they were likely icu_mortality but were marked as mortality_after_discharge due to inconsistencies in recording?

# Identify stay_ids where BOTH outcomes are 1
conflicting_stays = sf_mimicdata.loc[
    (sf_mimicdata["icu_mortality"] == 1) &
    (sf_mimicdata["mortality_within_7days_after_discharge"] == 1),
    "stay_id"
].unique()

# Set post-discharge mortality = 0 for those stay_ids
sf_mimicdata.loc[
    sf_mimicdata["stay_id"].isin(conflicting_stays),
    "mortality_within_7days_after_discharge"
] = 0


In [15]:
len(conflicting_stays)

117

In [16]:
#ratio of mortality
last_rows = sf_mimicdata.groupby("stay_id").tail(1)
(last_rows["mortality_within_7days_after_discharge"] == 1).mean()

0.0122

In [17]:
sf_mimicdata.stay_id.nunique()

5000

sf_mimicdata[sf_mimicdata.stay_id==30423672][['intime', 'outtime', 't', 'LOS']]

pd.to_datetime('2176-10-23 12:18:47') + pd.to_timedelta((19+ 1) * 12, unit="h")

time_to_discharge = pd.to_datetime('2176-11-02 12:27:33') - (pd.to_datetime('2176-10-23 12:18:47') + pd.to_timedelta((18+1) * 12, unit="h"))
time_to_discharge

time_to_discharge > pd.Timedelta(hours=6)

In [18]:
# Checking: (Recorded) discharge timestamp is close to (<=5hrs) to observation timestamp
### Remove last row if time difference is less than or equal to 5 hours

In [19]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [20]:
# Parameters
BUCKET_HOURS = 12
DECISION_HORIZON_HOURS = 5  # drop rows whose bucket-end is within 5h of ICU discharge

# Ensure datetimes
intime  = pd.to_datetime(sf_mimicdata["intime"],  errors="coerce")
outtime = pd.to_datetime(sf_mimicdata["outtime"], errors="coerce")

# Reference timestamp for each row = end of the bucket:
# t=0 -> 12h, t=1 -> 24h, etc.
ref_time = intime + pd.to_timedelta((sf_mimicdata["t"] + 1) * BUCKET_HOURS, unit="h")
sf_mimicdata["ref_time"] = ref_time

# Time remaining until discharge (can be negative if ref_time > outtime)
time_to_discharge = outtime - ref_time

# Keep rows where:
#  - outtime is missing (can't evaluate; keep), OR
#  - more than DECISION_HORIZON_HOURS remain until discharge
mask_keep = time_to_discharge.isna() | (
    time_to_discharge > pd.Timedelta(hours=DECISION_HORIZON_HOURS)
)

# (Optional) quick audit before dropping
dropped = (~mask_keep).sum()
kept = mask_keep.sum()
print(f"Filtering by {DECISION_HORIZON_HOURS}h decision horizon: drop {dropped} rows, keep {kept} rows.")

# Apply filter
sf_mimicdata = sf_mimicdata[mask_keep].reset_index(drop=True)


Filtering by 5h decision horizon: drop 6989 rows, keep 31838 rows.


In [21]:
sf_mimicdata.stay_id.nunique()

4539

# Add A, Y and D

In [22]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [23]:
sf_mimicdata.loc[:, 'A'] = np.nan
sf_mimicdata.loc[:, 'D'] = 0        #[Disputed. Yet to think. Only when A=0 AND (icu_mortality occurs, D=1 or no icu mortality, D=0). If A=1, D=0 (is it undefined?np.nan)
sf_mimicdata.loc[:, 'Y'] = np.nan

In [25]:
# Fill in A
sf_mimicdata["A"] = 0

# Update "A" to 0 if "icu_mortality" is True, and A=1 in other case (~ icu_mortality) at the last row of each 'stay_id'
sf_mimicdata.loc[sf_mimicdata.groupby('stay_id').tail(1).index, 'A'] = sf_mimicdata.groupby('stay_id').tail(1)['icu_mortality'].apply(lambda x: 0 if x else 1)

In [ ]:
# filter rows where icu_mortality = 1
subset = sf_mimicdata[sf_mimicdata["icu_mortality"] == 0]

In [ ]:
# pick one at random
random_stay = subset.sample(1)  # remove random_state for true random
print(random_stay["stay_id"].iloc[0])

In [ ]:
sf_mimicdata[sf_mimicdata.stay_id==31692426]

In [26]:
# Fill in D and Y

# A = 0 (continued ICU care)
sf_mimicdata.loc[(sf_mimicdata['A'] == 0) & (sf_mimicdata['icu_mortality'] == False), ['Y', 'D']] = [np.nan, 0]

#sf_mimicdata.loc[(sf_mimicdata['A'] == 0) & (sf_mimicdata['icu_mortality'] == True),  ['Y', 'D']] = [np.nan, 1]
# Update "D" to 1 if "icu_mortality" is True at the last row of each 'stay_id'
sf_mimicdata.loc[sf_mimicdata.groupby('stay_id').tail(1).index, 'D'] = sf_mimicdata.groupby('stay_id').tail(1)['icu_mortality'].apply(lambda x: 1 if x else 0)

# A = 1 (discharged)
sf_mimicdata.loc[(sf_mimicdata['A'] == 1) & (sf_mimicdata['mortality_within_7days_after_discharge'] == False), ['Y', 'D']] = [0, 0] #icu_mortality=0. Hence, D can be defined as 0?
sf_mimicdata.loc[(sf_mimicdata['A'] == 1) & (sf_mimicdata['mortality_within_7days_after_discharge'] == True),  ['Y', 'D']] = [1, 0] # Same as above


In [27]:
sf_mimicdata

,stay_id,grid_end,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,po2_arterial__mean__last_12h,...,death_time_from_intime,icu_mortality,death_abs_time,mortality_within_7days_after_discharge,vent_mode__hours_since_last__last_12h,t,ref_time,A,D,Y
0,mimic4-30002548,0 days 12:00:00,36.622223,79.230766,71.000000,225.000000,43.166668,13.384615,98.461540,306.166656,...,NaT,0,NaT,0,0,0,2111-08-18 01:13:43,0,0,NaN
1,mimic4-30002548,1 days 00:00:00,36.944447,70.500000,79.571426,87.500000,NaN,18.250000,98.666664,NaN,...,NaT,0,NaT,0,12,1,2111-08-18 13:13:43,1,0,0.0
2,mimic4-30003372,0 days 12:00:00,37.407410,109.083336,73.000000,70.699997,NaN,19.166666,98.000000,NaN,...,NaT,0,NaT,0,0,0,2170-06-10 15:15:10,0,0,NaN
3,mimic4-30003372,1 days 00:00:00,37.597221,110.384613,69.384613,93.181816,NaN,21.153847,95.923080,NaN,...,NaT,0,NaT,0,12,1,2170-06-11 03:15:10,0,0,NaN
4,mimic4-30003372,1 days 12:00:00,37.180557,100.083336,75.333336,131.428574,NaN,18.416666,96.000000,NaN,...,NaT,0,NaT,0,24,2,2170-06-11 15:15:10,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31833,mimic4-39997955,2 days 12:00:00,36.685188,65.750000,94.500000,168.750000,NaN,22.250000,97.500000,NaN,...,NaT,0,NaT,0,48,4,2160-10-29 07:03:02,0,0,NaN
31834,mimic4-39997955,3 days 00:00:00,36.907406,69.083336,95.916664,230.000000,NaN,21.166666,97.166664,NaN,...,NaT,0,NaT,0,60,5,2160-10-29 19:03:02,0,0,NaN
31835,mimic4-39997955,3 days 12:00:00,36.777779,70.500000,89.833336,245.714279,NaN,18.166666,96.416664,NaN,...,NaT,0,NaT,0,72,6,2160-10-30 07:03:02,0,0,NaN
31836,mimic4-39997955,4 days 00:00:00,36.861115,80.000000,98.416664,209.375000,NaN,19.000000,96.833336,NaN,...,NaT,0,NaT,0,84,7,2160-10-30 19:03:02,0,0,NaN


sf_mimicdata['all_M'] = ((sf_mimicdata['mortality_within_7days_after_discharge'].fillna(0) == 1) | (sf_mimicdata['icu_mortality'].fillna(0) == 1)).astype(int)

In [28]:
sf_mimicdata = sf_mimicdata.reset_index(drop=True)

sf_mimicdata = sf_mimicdata.rename(columns={
    'Y': 'Y_old',
    'all_M': 'Y'
})

In [29]:
sf_mimicdata.groupby('stay_id').tail(1)['Y'].mean()

0.015011258443832875

In [30]:
sf_mimicdata.groupby('stay_id').tail(1)['D'].mean()

0.11940956157743997

In [31]:
sf_mimicdata.to_csv('../Dataset/mimic-iv/from_pipeline/ltm_grided_cleaned_5000_A_Y_D_SelfPipeline.parquet')

In [32]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'temperature__mean__last_12h',
       'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'age', 'sex', 'height', 'weight',
       'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_within_7days_after_discharge',
       'vent_mode__hours_since_last__last_12h', 't', 'ref_time', 'A', 'D',
       'Y'],
